In [1]:
from Crypto.Cipher import ChaCha20
from Crypto.Random import get_random_bytes
import cv2

def chacha20_encrypt(data, key, iv=None, position=0):
    if not isinstance(data, bytes):
        raise TypeError('Data must be bytes.')
    if iv is None:
        iv = b'\0' * 8
    if isinstance(key, bytes):
        if not key:
            raise ValueError('Key is empty.')
        if len(key) < 32:
            key = (key * (32 // len(key) + 1))[:32]
        if len(key) > 32:
            raise ValueError('Key is too long.')

    # Initialize ChaCha20 cipher
    cipher = ChaCha20.new(key=key, nonce=iv)

    encrypted_data = b''
    # Encrypt data in blocks
    for i in range(0, len(data), 4096):
        block = data[i:i+4096]
        encrypted_data += cipher.encrypt(block)

    return encrypted_data

def chacha20_decrypt(data, key, iv=None, position=0):
    decrypted_data = chacha20_encrypt(data, key, iv, position)
    return decrypted_data

def encrypt_image(input_path, output_path, key):
    with open(input_path, 'rb') as image_file:
        image_data = image_file.read()

    iv = get_random_bytes(8)
    encrypted_data = chacha20_encrypt(image_data, key, iv)

    with open(output_path, 'wb') as encrypted_file:
        encrypted_file.write(iv + encrypted_data)

def decrypt_image(input_path, output_path, key):
    with open(input_path, 'rb') as encrypted_file:
        data = encrypted_file.read()

    iv = data[:8]
    encrypted_data = data[8:]

    decrypted_data = chacha20_decrypt(encrypted_data, key, iv)

    with open(output_path, 'wb') as decrypted_file:
        decrypted_file.write(decrypted_data)

# Paths
input_image_path = 'shuffled.jpg'
encrypted_image_path = 'encrypted_image.png'
decrypted_image_path = 'decrypted_image.png'

# Generate a random key
key = get_random_bytes(32)  # Generating a random 256-bit key

# Encrypt image
encrypt_image(input_image_path, encrypted_image_path, key)

# Decrypt image
decrypt_image(encrypted_image_path, decrypted_image_path, key)

# Load decrypted image
decrypted_image = cv2.imread(decrypted_image_path)

if decrypted_image is None:
    print("Error: Failed to load the decrypted image.")
else:
    cv2.imshow("Decrypted Image", decrypted_image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


KeyboardInterrupt: 